In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
import psycopg2
from sqlalchemy import create_engine



In [6]:
# Informations de connexion
HOST = "postgresql-rospars.alwaysdata.net"
USER = "rospars_01"
PASSWORD = "Projet1234"
DATABASE = "rospars_yann"
PORT = 5432  # Port par défaut de PostgreSQL


In [7]:
# Créer l'engine SQLAlchemy
engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")
# Récupérer les films depuis la base
query = "SELECT id, title, overview, popularity, vote_average FROM movies;"
movies_df = pd.read_sql(query, engine)

# Vérifier les données récupérées
print(movies_df.head())


      id                        title  \
0    862                    Toy Story   
1   8844                      Jumanji   
2  15602             Grumpier Old Men   
3  31357            Waiting to Exhale   
4  11862  Father of the Bride Part II   

                                            overview  popularity  vote_average  
0  Led by Woody, Andy's toys live happily in his ...   21.946943           7.7  
1  When siblings Judy and Peter discover an encha...   17.015539           6.9  
2  A family wedding reignites the ancient feud be...   11.712900           6.5  
3  Cheated on, mistreated and stepped on, the wom...    3.859495           6.1  
4  Just when George Banks has recovered from his ...    8.387519           5.7  


In [8]:
# Vérifier les valeurs manquantes
print(movies_df.isnull().sum())

# Supprimer les films sans description
movies_df.dropna(subset=["overview"], inplace=True)

# Convertir `popularity` et `vote_average` en float (si nécessaire)
movies_df["popularity"] = movies_df["popularity"].astype(float)
movies_df["vote_average"] = movies_df["vote_average"].astype(float)

# Vérifier les changements
print(movies_df.info())

id              0
title           0
overview        0
popularity      0
vote_average    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45351 entries, 0 to 45350
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            45351 non-null  int64  
 1   title         45351 non-null  object 
 2   overview      45351 non-null  object 
 3   popularity    45351 non-null  float64
 4   vote_average  45351 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.7+ MB
None


Charger les genres et mots-clés

In [9]:
engine.dispose()
# Charger les genres
query_genres = "SELECT id, name FROM genres;"
genres_df = pd.read_sql(query_genres, engine)

# Charger les mots-clés
query_keywords = "SELECT id, name FROM keywords;"
keywords_df = pd.read_sql(query_keywords, engine)


In [10]:
print(genres_df.head())
print(keywords_df.head())


      id       name
0     16  Animation
1     35     Comedy
2  10751     Family
3     12  Adventure
4     14    Fantasy
     id        name
0   931    jealousy
1  4290         toy
2  5202         boy
3  6054  friendship
4  9713     friends


 Charger les tables de correspondance

In [11]:
# Charger la table movies_genre
query_movies_genre = "SELECT movie_id, genre_id FROM movie_genre;"
movies_genre_df = pd.read_sql(query_movies_genre, engine)

# Charger la table movies_keyword
query_movies_keyword = "SELECT movie_id, keyword_id FROM movie_keyword;"
movies_keyword_df = pd.read_sql(query_movies_keyword, engine)

# Vérifier les données
print(movies_genre_df.head())
print(movies_keyword_df.head())

   movie_id  genre_id
0       862        16
1       862        35
2       862     10751
3      8844        12
4      8844        14
   movie_id  keyword_id
0       862         931
1       862        4290
2       862        5202
3       862        6054
4       862        9713


Associer genres et mots-clés aux films

In [12]:
# Fusionner movies_genre avec genre pour récupérer le nom du genre
movies_genre_df = movies_genre_df.merge(genres_df, left_on="genre_id", right_on="id", how="left")
movies_genre_df = movies_genre_df.groupby("movie_id")["genre"].apply(lambda x: " ".join(x)).reset_index()

# Fusionner movies_keyword avec keywords pour récupérer le nom du mot-clé
movies_keyword_df = movies_keyword_df.merge(keywords_df, left_on="keyword_id", right_on="id", how="left")
movies_keyword_df = movies_keyword_df.groupby("movie_id")["keyword"].apply(lambda x: " ".join(x)).reset_index()

KeyError: 'Column not found: genre'

In [ ]:
print(movies_keyword_df.head())
print(movies_genre_df.head())



   movie_id  keyword_id
0       862         931
1       862        4290
2       862        5202
3       862        6054
4       862        9713
   movie_id  genre_id
0       862        16
1       862        35
2       862     10751
3      8844        12
4      8844        14


**Créer un DataFrame `movies_metadata_df`**

dédié aux relations movie_id - genre - keyword

In [ ]:
# Fusionner les genres et keywords sur movie_id
movies_metadata_df = movies_genre_df.merge(movies_keyword_df, on="movie_id", how="outer")

# Remplacer les NaN par des chaînes vides
movies_metadata_df.fillna("", inplace=True)

# Afficher un aperçu
print(movies_metadata_df.head())


   movie_id  genre_id keyword_id
0         2        18      240.0
1         2        18      378.0
2         2        18      730.0
3         2        18     1563.0
4         2        18     1787.0


C:\Users\PC-Service\AppData\Local\Temp\ipykernel_27068\1745719976.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  movies_metadata_df.fillna("", inplace=True)
